# 0. Pre-processing

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ubi_sales_df = pd.read_csv('Ubisoft_game_sales.csv', parse_dates=['ReleaseDate'])

In [ ]:
ubi_sales_df.info()

In [ ]:
date_parser = lambda date: pd.datetime.strptime(date, '%b %d, %Y')

In [ ]:
ubi_rates_df = pd.read_csv('Ubisoft_game_rates.csv')

In [ ]:
ubi_rates_df = ubi_rates_df[ubi_rates_df['Score'] != 'tbd']\
                           [np.logical_not(ubi_rates_df['Date'].isin(['TBA', 'Canceled']))]

In [ ]:
# 转换数据类型
ubi_rates_df['Score'] = pd.to_numeric(ubi_rates_df['Score'])
ubi_rates_df['Date'] = pd.to_datetime(ubi_rates_df['Date'])

In [ ]:
ubi_rates_df.info()

In [ ]:
import re

In [ ]:
# pattern to capture platform
ptn_1 = re.findall(r'(\(\w+\))$', 'Starlink:(PS4) Battle for Atlas (PS4)')[0][1:-1]

In [ ]:
# pattern to remove platform
ptn_2 = re.sub(r' (\(\w+\))$', '', 'Starlink:(PS4) Battle for Atlas (PS4)')

In [ ]:
ubi_rates_df['Platform'] = ubi_rates_df['Title'].apply(lambda x: re.findall(r'(\(\w+\))$', x)[0][1:-1])

In [ ]:
ubi_rates_df['Title'] = ubi_rates_df['Title'].apply(lambda x: re.sub(r' (\(\w+\))$', '', x))

In [ ]:
ubi_rates_df.to_csv('Ubisoft_game_rates_v2.csv')

In [ ]:
# 统一游戏名称
def unite_name(name):
    x = re.sub(r'[\:\/\.]', '', name.lower())
    return re.sub(r'\s', '-', x)

In [ ]:
# 去除特殊字符
ubi_sales_df['Name'].loc[12] = "Assassin's Creed Rogue Remastered" # 一个爬虫抓取错误
ubi_sales_df['Name'] = ubi_sales_df['Name'].apply(unite_name)

In [ ]:
ubi_rates_df['Title'] = ubi_rates_df['Title'].apply(unite_name)

# 1. Merge DataFrame

In [ ]:
ubi_rates = ubi_rates_df.groupby('Title').max()

In [ ]:
ubi_sales = ubi_sales_df.groupby('Name').sum()

In [ ]:
ubi_rates['GlobalSale'] =  [ubi_sales.loc[game]['GlobalSale'] 
                            if game in ubi_sales.index else np.nan 
                            for game in ubi_rates.index]

In [ ]:
ubi_rates = ubi_rates[ubi_rates['GlobalSale'].notnull()]

In [ ]:
ubi_rates.drop('Platform', axis=1, inplace=True)

In [ ]:
ubi_rates = ubi_rates.sort_values(['Date', 'GlobalSale', 'Score'])

In [ ]:
ubi_rates.to_csv('Ubisoft_game_rates_v3.csv')

In [ ]:
ubi_rates.info()

# 2. Visualization

In [ ]:
ubi_rates = pd.read_csv('Ubisoft_game_rates_v3.csv', parse_dates=['Date'])

In [ ]:
# Stock data
ubi_stocks = pd.read_csv('UBI.PA.MONTHLY.csv', 
                         parse_dates=['Date'], index_col='Date')

In [ ]:
ubi_stocks.head()

In [ ]:
ubi_stock_dates = ubi_stocks.index.to_native_types()

In [ ]:
ubi_prices = ubi_stocks[['Open', 'Close', 'Low', 'High']].values.tolist()

In [ ]:
ubi_release_dates = ubi_rates['Date'].apply(lambda x: x.strftime('%Y-%m-%d')).values.tolist()

In [ ]:
ubi_names = ubi_rates['Title'].tolist()

In [ ]:
ubi_scores =  ubi_rates['Score'].tolist()

In [ ]:
ubi_sales = ubi_rates['GlobalSale'].tolist()

In [ ]:
from pyecharts import Scatter, Line, Kline, Overlap

In [ ]:
def custom_formatter(params):
    return params.value[3]

In [ ]:
sc = Scatter('Ubi game rating')

In [ ]:
sc.add("Game rating",
        ubi_release_dates,
        ubi_scores,
        extra_data=ubi_sales,
        extra_name=ubi_names,
        is_visualmap=True,
        is_datazoom_show=True,
        datazoom_xaxis_index=[0, 1],
        xaxis_type='time',
        xaxis_min = '1999-12-31',
        xaxis_max = '2018-11-05',
        symbol_size=8,
        alpha=0.5,
        visual_dimension=2,
        visual_range=[0, 10],
        tooltip_formatter=custom_formatter)

In [ ]:
kl = Kline('Ubi stock prices')

In [ ]:
kl.add('Stock price', ubi_stock_dates, ubi_prices,
      is_datazoom_show=True, 
      is_splitline_show=False, 
      yaxis_max=150)

In [ ]:
ov = Overlap()

In [ ]:
ov.add(sc)
ov.add(kl, xaxis_index=1, yaxis_index=1,
          is_add_xaxis=True, is_add_yaxis=True)

In [ ]:
ov.show_config()